In [5]:
import psycopg2
import sys
from sqlalchemy import create_engine
import geopandas as gpd
import os

In [6]:
DATABASE_NAME = "lab_postgis"
PASSWORD =  os.environ.get("POSTGRES_PASS")

In [7]:
def getEngineConnectionString(user='postgres', password=PASSWORD, dbname='postgres'):
    return "postgresql://" + user + ":" + password + "@localhost/" + dbname

In [8]:
def getOpenConnection(user='postgres', password=PASSWORD, dbname='postgres'):
    return psycopg2.connect(database = dbname, user = user, host='localhost', password= password)

In [9]:
def createDB(dbname='postgres'):
    """
    We create a DB by connecting to the default user and database of Postgres
    The function first checks if an existing database exists for a given name, else creates it.
    :return:None
    """
    # Connect to the default database
    con = getOpenConnection(dbname='postgres')
    con.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = con.cursor()

    # Check if an existing database with the same name exists
    cur.execute('SELECT COUNT(*) FROM pg_catalog.pg_database WHERE datname=\'%s\'' % (dbname,))
    count = cur.fetchone()[0]
    if count == 0:
        cur.execute('CREATE DATABASE %s' % (dbname,))  # Create the database
        print("DATABASE " + dbname + " created")
    else:
        print("Database with the given name already exists")

    # Clean up
    cur.close()
    con.commit()
    con.close()

In [10]:
createDB(DATABASE_NAME)

Database with the given name already exists


In [11]:
engine = create_engine(getEngineConnectionString(dbname = DATABASE_NAME))
con = engine.connect()
print("sqlalchemy engine created")

sqlalchemy engine created


In [12]:
try:
    con.execute("CREATE EXTENSION postgis;") # Add PostGIS extension
    print("postgis extension created")
except:
    print("Unable to create postgis extension")

Unable to create postgis extension


In [13]:
dirAirbnb = gpd.read_file("inputs/airbnb")

In [14]:
dirAirbnb.head()

,community,shape_area,shape_len,AREAID,response_r,accept_r,rev_rating,price_pp,room_type,num_spots,...,crowded,dependency,without_hs,unemployed,income_pc,harship_in,num_crimes,num_theft,population,geometry
0,DOUGLAS,46004621.1581,31027.0545098,35,98.771429,94.514286,87.777778,78.157895,1.789474,38,...,1.8,30.7,14.3,18.2,23791,47,5013,1241,18238,"POLYGON ((-87.60914 41.84469, -87.60915 41.844..."
1,OAKLAND,16913961.0408,19565.5061533,36,99.200000,90.105263,88.812500,53.775000,1.850000,20,...,1.3,40.4,18.4,28.7,19252,78,1306,311,5918,"POLYGON ((-87.59215 41.81693, -87.59231 41.816..."
2,FULLER PARK,19916704.8692,None,37,68.000000,NaN,91.750000,84.000000,1.833333,6,...,3.2,44.9,26.6,33.9,10432,97,1764,383,2876,"POLYGON ((-87.62880 41.80189, -87.62879 41.801..."
3,GRAND BOULEVARD,48492503.1554,28196.8371573,38,94.037037,83.615385,92.750000,119.533333,1.533333,30,...,3.3,39.5,15.9,24.3,23472,57,6416,1428,21929,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
4,KENWOOD,29071741.9283,23325.1679062,39,92.542857,88.142857,90.656250,77.991453,1.615385,39,...,2.4,35.4,11.3,15.7,35911,26,2713,654,17841,"POLYGON ((-87.59215 41.81693, -87.59215 41.816..."


In [15]:
dirAirbnb.to_postgis("airbnb", engine)

In [18]:
dfGrocery = gpd.read_file("inputs/grocery/chicago_sup.shp")

In [19]:
dfGrocery.head()


,OBJECTID,Ycoord,Xcoord,Status,Address,Chain,Category,geometry
0,16,41.973266,-87.657073,OPEN,"1051 W ARGYLE ST, CHICAGO, IL. 60640",VIET HOA PLAZA,None,MULTIPOINT (1168268.672 1933554.350)
1,18,41.696367,-87.681315,OPEN,"10800 S WESTERN AVE, CHICAGO, IL. 60643-3226",COUNTY FAIR FOODS,None,MULTIPOINT (1162302.618 1832900.224)
2,22,41.868634,-87.638638,OPEN,"1101 S CANAL ST, CHICAGO, IL. 60607-4932",WHOLE FOODS MARKET,None,MULTIPOINT (1173317.042 1895425.426)
3,23,41.877590,-87.654953,OPEN,"1101 W JACKSON BLVD, CHICAGO, IL. 60607-2905",TARGET/SUPER,new,MULTIPOINT (1168996.475 1898801.406)
4,27,41.737696,-87.625795,OPEN,"112 W 87TH ST, CHICAGO, IL. 60620-1318",FOOD 4 LESS,None,MULTIPOINT (1176991.989 1847262.423)


In [20]:
dfGrocery.to_postgis("grocery" , engine)

In [51]:
if con:
    con.close()

In [52]:
con = getOpenConnection(dbname = DATABASE_NAME)

In [53]:
cur = con.cursor()

In [54]:
cur.execute("select ST_SRID(geometry) from airbnb limit 1")
result = cur.fetchall()
print(result)

[(4326,)]


In [55]:
cur.execute("select ST_SRID(geometry) from grocery limit 1")
result = cur.fetchall()
print(result)

[(6312,)]


In [56]:
cur.execute("create table grocery2 as select ST_Transform(geometry, 4326) as geometry from grocery")

In [57]:
cur.execute("select ST_SRID(geometry) from grocery2 limit 1")
result = cur.fetchall()
print(result)

[(4326,)]


In [58]:
 cur.execute("CREATE TABLE airbnb_grocery2 AS SELECT airbnb.geometry AS geom1, COUNT(grocery2.geometry) as grocery_count FROM airbnb, grocery2 WHERE ST_DISTANCE(airbnb.geometry, grocery2.geometry) < 133.9 GROUP BY airbnb.geometry")

In [61]:
cur.execute("SELECT * FROM airbnb_grocery2 limit 5")
result = cur.fetchall()
print(result)

[('0103000020E6100000010000005F0200008CA9BDD001E755C006147FD4D7F14440C7F299CF01E755C055880488D7F14440F601E265E4E655C00ED209EED6F144403A6F0058E4E655C0E906DBF0D6F14440CEE59228E4E655C037312FEED6F14440A0B317FBE3E655C09E055AD9D6F14440DBE0CAD2E3E655C0C429C8B3D6F14440F96146B2E3E655C04DDC4280D6F14440E281AD9BE3E655C041AFD241D6F144407227DC84E3E655C01C3B2A09D4F14440C2B5CF8EE3E655C087298CDDD3F14440FAEA719DE3E655C03D90A9B6D3F14440B7FEEEA8E3E655C0500A2298D3F14440AA6B00CDE3E655C00F346D5DD3F144408F1B28F9E3E655C01D46A130D3F144407FDD63ABE4E655C02D75482ED3F144409F93FA51E4E655C058E75092D2F1444068A95E16E4E655C0E3EA562AD2F14440DE67F8C9E3E655C0D521D315C0F144403ED2515BE2E655C00F823D1CC0F14440A7108D82E1E655C017D872F7BEF144407EED233BC7E655C0E257D6C1BFF14440B086A747C3E655C086642FCDBFF144403ED795EABFE655C0B28E5A01C0F1444057000DADBFE655C07A547004C0F14440D27B6C80BFE655C0F54AAD06C0F144405F270A6CBFE655C04E656F3AC1F144402863DF41BCE655C0A10B9562C1F14440CC6B33EABBE655C0DAA0FD66C1F1444059748792BBE655C0FC33666BC1F144403B3